In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [3]:
from glob import glob
import librosa
import librosa.display 

In [4]:
# 데이터 불러오기 


africa_list = glob("./data/train/africa/*.wav")#glob 함수는 입력하는 폴더 아래 파일들의 path를 가져온다.
australia_list = glob("./data/train/australia/*.wav")#glob 함수는 입력하는 폴더 아래 파일들의 path를 가져온다.
canada_list = glob("./data/train/canada/*.wav")#glob 함수는 입력하는 폴더 아래 파일들의 path를 가져온다.
england_list = glob("./data/train/england/*.wav")#glob 함수는 입력하는 폴더 아래 파일들의 path를 가져온다.
hongkong_list = glob("./data/train/hongkong/*.wav")#glob 함수는 입력하는 폴더 아래 파일들의 path를 가져온다.
us_list = glob("./data/train/us/*.wav")#glob 함수는 입력하는 폴더 아래 파일들의 path를 가져온다.


X_list = africa_list + canada_list + england_list +australia_list +hongkong_list + us_list
Y_list = [0]*len(africa_list)+[1]*len(australia_list) + [2]*len(canada_list)+ [3]*len(england_list)+ [4]*len(hongkong_list)+ [5]*len(us_list)
# 라벨명 :  아프리카: 0 , 호주: 1, 캐나다: 2 , 영국: 3, 홍콩: 4,   미국: 5
 

In [5]:
# 오디오 데이터로 만들어 주기.
x_list_audio = [ librosa.load(x)[0] for x in X_list]

In [6]:
#문제. 각 데이터마다 이미지 사이즈가 다르다.
# 처음부터 읽어올때 시그널 사이즈를 맞춰주자!
# 5초 정도로 자르기. 5초보다 짧은 음성은 0으로 채워주기. padding이라고 부른다.

#패딩 함수
length = 22050*5 # 5초 사이즈 padding을 위한 파라미터 
pad1d = lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))        
#sig = pad1d(sig,length) 
x_list_audio = [ pad1d(librosa.load(x)[0],length) for x in X_list]






In [7]:
sr =22050
win_length =  np.int64(sr/40) # 
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다. (stride)
#hop: 윈도우가 겹치는 사이즈

# mel spectrogram으로 변환
x_list_audio = [librosa.feature.melspectrogram(y=x, sr=sr,win_length=win_length ,n_fft=n_fft, hop_length=hop_length) for x in x_list_audio ] 
x_list_audio = [librosa.core.power_to_db(x,ref=np.max) for x in x_list_audio ] 


x_list_audio = np.array(x_list_audio) # 리스트를 numpy array로


In [8]:
print(x_list_audio.shape)
# (batch,컬럼) 형태의 구조
#x_train_audio


#128 *799 사이즈의 1296개 이미지가 생성된다.

(1296, 128, 799)


In [9]:
#validation set 두기 (정확한 평가를 위해서)
#주로 하이퍼 파라미터 찾기 위해서 validation set을 둔다.

from sklearn.model_selection import train_test_split

x_train,x_validate,y_train,y_validate = train_test_split(x_list_audio,Y_list, test_size = 0.2,random_state = 12345)


#y label들 numpy 바꿔주기 
y_train= np.array(y_train)
y_validate= np.array(y_validate)

print(x_train.shape, x_validate.shape)


(1036, 128, 799) (260, 128, 799)


In [10]:
# 이미지는 3채널이어야 하기에 rgb 3채널로 복사해주기

x_train = np.repeat(x_train[:, :, :, np.newaxis], 3, axis=3)
x_validate = np.repeat(x_validate[:, :, :, np.newaxis], 3, axis=3)



In [11]:
print(x_train.shape, x_validate.shape)


(1036, 128, 799, 3) (260, 128, 799, 3)


In [12]:
#모델 구축
#https://keras.io/api/applications/
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50




res_model = ResNet50(input_shape=(128,799, 3),
                              include_top=False,
                              weights="imagenet")

res_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 799, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 805, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 400, 64)  9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 400, 64)  256         ['conv1_conv[0][0]']      

In [13]:
import numpy as np
res_model(np.random.randn(1,128,799,3)).shape

TensorShape([1, 4, 25, 2048])

In [14]:
#모델 구축
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,Activation,Flatten

def resnet50_model():
    pretrained_model = res_model
    model = Sequential()

    # Add the vgg convolutional base model
    model.add(pretrained_model)

    # Add new layers
    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))   
    model.add(Dense(256, activation='relu'))
    model.add(Dense(6, activation='softmax')) # 6개의 국가로 나눠야하기 때문. softmax함수가 7개의 국가일 확률을 구해준다.
    return model

nn_model= resnet50_model()

In [15]:
#compile이란 우리가 만든 모델을 어떻게 weight들을 최적화 시킬지 정해주는 부분.
from keras.optimizers import Adam
nn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics =['accuracy'])
#Adam은 가중치를 학습 시켜주는 optimizer중 가장 유명한 것.
#learning rate는 데이터, 전처리 방법에 따라 달라진다. 스스로 조절해가며 찾아야한다.
#metrics는 학습하면서 평가해주는 중간결과를 보여줄 때 사용.

#sparse_categorical_crossentropy는 0~10으로된 수치형 라벨을 10개로 만들어진 백터와 매칭시켜준다.

#categorical_crossentropy를 쓰면,0~10칸으로 된 라벨을 10개 벡터와 매칭.


In [16]:
nn_model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=10,
    verbose=1,
    validation_data=(x_validate,y_validate),
)

Epoch 1/10
65/65 [==============================] - 14s 155ms/step - loss: 12.0955 - accuracy: 0.2780 - val_loss: 4191.4429 - val_accuracy: 0.1808
Epoch 2/10
65/65 [==============================] - 9s 131ms/step - loss: 1.4586 - accuracy: 0.4054 - val_loss: 33.0027 - val_accuracy: 0.1962
Epoch 3/10
65/65 [==============================] - 9s 133ms/step - loss: 1.1586 - accuracy: 0.5685 - val_loss: 17.1967 - val_accuracy: 0.2000
Epoch 4/10
65/65 [==============================] - 9s 132ms/step - loss: 0.9634 - accuracy: 0.6660 - val_loss: 1.6618 - val_accuracy: 0.4462
Epoch 5/10
65/65 [==============================] - 8s 130ms/step - loss: 0.6495 - accuracy: 0.7790 - val_loss: 1.6053 - val_accuracy: 0.4077
Epoch 6/10
65/65 [==============================] - 9s 134ms/step - loss: 0.5830 - accuracy: 0.8050 - val_loss: 1.4957 - val_accuracy: 0.6000
Epoch 7/10
65/65 [==============================] - 9s 132ms/step - loss: 0.4538 - accuracy: 0.8523 - val_loss: 2.3842 - val_accuracy: 0.5654

In [78]:
#from numba import cuda
#cuda.select_device(0)
#cuda.close()

# 테스트 코드 작성

In [17]:
# 테스트 데이터 읽어서 mel spectrogram으로 만들기

test_list = glob("./data/test/*.wav")#glob 함수는 입력하는 폴더 아래 파일들의 path를 가져온다.
#단, test파일이 1~60000으로 정렬 되어있는지 확인하기.

In [18]:
test_list

['./data/test\\1.wav',
 './data/test\\10.wav',
 './data/test\\100.wav',
 './data/test\\1000.wav',
 './data/test\\1001.wav',
 './data/test\\1002.wav',
 './data/test\\1003.wav',
 './data/test\\1004.wav',
 './data/test\\1005.wav',
 './data/test\\1006.wav',
 './data/test\\1007.wav',
 './data/test\\1008.wav',
 './data/test\\1009.wav',
 './data/test\\101.wav',
 './data/test\\1010.wav',
 './data/test\\1011.wav',
 './data/test\\1012.wav',
 './data/test\\1013.wav',
 './data/test\\1014.wav',
 './data/test\\1015.wav',
 './data/test\\1016.wav',
 './data/test\\1017.wav',
 './data/test\\1018.wav',
 './data/test\\1019.wav',
 './data/test\\102.wav',
 './data/test\\1020.wav',
 './data/test\\1021.wav',
 './data/test\\1022.wav',
 './data/test\\1023.wav',
 './data/test\\1024.wav',
 './data/test\\1025.wav',
 './data/test\\1026.wav',
 './data/test\\1027.wav',
 './data/test\\1028.wav',
 './data/test\\1029.wav',
 './data/test\\103.wav',
 './data/test\\1030.wav',
 './data/test\\1031.wav',
 './data/test\\1032.w

In [19]:
test_list = sorted(test_list,key=lambda x: int(x.split('\\')[1].split('.')[0]))
test_list

['./data/test\\1.wav',
 './data/test\\2.wav',
 './data/test\\3.wav',
 './data/test\\4.wav',
 './data/test\\5.wav',
 './data/test\\6.wav',
 './data/test\\7.wav',
 './data/test\\8.wav',
 './data/test\\9.wav',
 './data/test\\10.wav',
 './data/test\\11.wav',
 './data/test\\12.wav',
 './data/test\\13.wav',
 './data/test\\14.wav',
 './data/test\\15.wav',
 './data/test\\16.wav',
 './data/test\\17.wav',
 './data/test\\18.wav',
 './data/test\\19.wav',
 './data/test\\20.wav',
 './data/test\\21.wav',
 './data/test\\22.wav',
 './data/test\\23.wav',
 './data/test\\24.wav',
 './data/test\\25.wav',
 './data/test\\26.wav',
 './data/test\\27.wav',
 './data/test\\28.wav',
 './data/test\\29.wav',
 './data/test\\30.wav',
 './data/test\\31.wav',
 './data/test\\32.wav',
 './data/test\\33.wav',
 './data/test\\34.wav',
 './data/test\\35.wav',
 './data/test\\36.wav',
 './data/test\\37.wav',
 './data/test\\38.wav',
 './data/test\\39.wav',
 './data/test\\40.wav',
 './data/test\\41.wav',
 './data/test\\42.wav',
 

In [20]:
#문제. 각 데이터마다 이미지 사이즈가 다르다.
# 처음부터 읽어올때 시그널 사이즈를 맞춰주자!
# 5초 정도로 자르기. 5초보다 짧은 음성은 0으로 채워주기. padding이라고 부른다.

#패딩 함수
length = 22050*5 # 5초 사이즈 padding을 위한 파라미터 
pad1d = lambda a, i: a[0:i] if a.shape[0] > i else np.hstack((a, np.zeros((i-a.shape[0]))))        
#sig = pad1d(sig,length) 
x_test_audio = [ pad1d(librosa.load(x)[0],length) for x in test_list]


In [21]:

sr =22050
win_length =  np.int64(sr/40) # 
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다. (stride)
#hop: 윈도우가 겹치는 사이즈

# mel spectrogram으로 변환
x_test_audio = [librosa.feature.melspectrogram(y=x, sr=sr,win_length=win_length ,n_fft=n_fft, hop_length=hop_length) for x in x_test_audio ] 
x_test_audio = [librosa.core.power_to_db(x,ref=np.max) for x in x_test_audio ] 


x_test_audio = np.array(x_test_audio) # 리스트를 numpy array로

In [22]:
# 이미지는 3채널이어야 하기에 rgb 3채널로 복사해주기

x_test_audio = np.repeat(x_test_audio[:, :, :, np.newaxis], 3, axis=3)


In [72]:
x_test_audio.shape

array([[[[-80.      , -80.      , -80.      ],
         [-80.      , -80.      , -80.      ],
         [-80.      , -80.      , -80.      ],
         ...,
         [-55.97033 , -55.97033 , -55.97033 ],
         [-60.96189 , -60.96189 , -60.96189 ],
         [-55.961475, -55.961475, -55.961475]],

        [[-80.      , -80.      , -80.      ],
         [-80.      , -80.      , -80.      ],
         [-80.      , -80.      , -80.      ],
         ...,
         [-55.072906, -55.072906, -55.072906],
         [-60.064472, -60.064472, -60.064472],
         [-55.064056, -55.064056, -55.064056]],

        [[-80.      , -80.      , -80.      ],
         [-80.      , -80.      , -80.      ],
         [-80.      , -80.      , -80.      ],
         ...,
         [-45.722275, -45.722275, -45.722275],
         [-43.806168, -43.806168, -43.806168],
         [-44.035736, -44.035736, -44.035736]],

        ...,

        [[-80.      , -80.      , -80.      ],
         [-80.      , -80.      , -80.      ]

In [23]:
#메모리가 부족한 상황 때문에 2000개씩 나눠서 투입

y_pred1 = nn_model.predict(x_test_audio[:2000])

63/63 [==============================] - 4s 51ms/step


In [26]:
#메모리가 부족한 상황 때문에 2000개씩 나눠서 투입
y_pred2 = nn_model.predict(x_test_audio[2000:2000*2])

63/63 [==============================] - 3s 52ms/step


In [27]:
#메모리가 부족한 상황 때문에 2000개씩 나눠서 투입
y_pred3 = nn_model.predict(x_test_audio[2000*2:])

66/66 [==============================] - 3s 52ms/step


In [29]:
y_pred1.shape
#6칸에 대해, 확률들을 적어 둔 것.

(2000, 6)

In [30]:
y_pred1

array([[9.91447091e-01, 1.28490792e-03, 3.23151681e-03, 3.11536784e-03,
        4.10836568e-04, 5.10178739e-04],
       [4.48507664e-04, 1.36206208e-05, 2.91154091e-03, 4.57768887e-03,
        9.92015839e-01, 3.27787857e-05],
       [5.07604778e-01, 1.96584240e-02, 2.06180792e-02, 6.83906907e-03,
        1.96295534e-03, 4.43316668e-01],
       ...,
       [9.59108353e-01, 2.73326300e-02, 2.00661318e-03, 1.10856621e-02,
        1.13823946e-04, 3.52977222e-04],
       [9.92315036e-05, 9.99047458e-01, 4.39733703e-04, 2.75556580e-04,
        1.19682569e-04, 1.84080818e-05],
       [9.89243627e-01, 2.32167047e-04, 7.47034675e-04, 9.73202102e-03,
        9.77367472e-06, 3.53258183e-05]], dtype=float32)

In [31]:
submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8')

In [32]:
# 2000번까지 확률 입력
submission.iloc[:2000,1:] = y_pred1 # 예측 확률들을 다 집어넣는다.

In [33]:
# 2000~4000 확률 입력
submission.iloc[2000:2000*2,1:] = y_pred2 # 예측 확률들을 다 집어넣는다.

In [34]:
#4000 ~ 그외
submission.iloc[2000*2:,1:] = y_pred3 # 예측 확률들을 다 집어넣는다.

In [35]:
submission.to_csv('voice_submission.csv', index = False)

In [37]:
submission.shape

(6100, 7)

In [38]:

submission

,id,africa,australia,canada,england,hongkong,us
0,1,0.991447,1.284908e-03,0.003232,0.003115,4.108366e-04,5.101787e-04
1,2,0.000449,1.362062e-05,0.002912,0.004578,9.920158e-01,3.277879e-05
2,3,0.507605,1.965842e-02,0.020618,0.006839,1.962955e-03,4.433167e-01
3,4,0.999892,2.412206e-07,0.000096,0.000012,7.832716e-09,5.284309e-08
4,5,0.278975,4.281072e-05,0.002175,0.000209,7.105247e-01,8.073536e-03
...,...,...,...,...,...,...,...
6095,6096,0.004261,7.725934e-03,0.870812,0.005711,1.091760e-01,2.313862e-03
6096,6097,0.145086,7.477363e-02,0.754185,0.002669,1.519360e-02,8.093343e-03
6097,6098,0.526848,2.395072e-02,0.098310,0.012020,3.508757e-03,3.353619e-01
6098,6099,0.878183,5.144886e-03,0.019222,0.001225,9.569270e-02,5.336105e-04


점수를 높일 수 있는 시도들
1. 데이터셋을 모두 넣는다.
2. dacon baseline을 따라 작성해본다
3. 다른 코드들 따라한다.